Optimización de hiperparámetros

Amazon SageMaker nos ofrece la funcionalidad de optimización de hiperpárametros, ya sea utilizando una búsqueda aleatoria o un método bayesiano, en este caso vamos a utilizar el segundo método el cual permite entrenar un modelo de forma iterativa e ir identificando que combinación de hiperparámetros, nos permite minimizar o maximizar más la métrica objetivo.

Crear proceso de optimización (Tuning Job)

Gracias a que creamos tres estimators para el entrenamiento, ahora podemos crear tres procesos de optimización que se ejecuten en paralelo, para esto utilizamos la clase HyperparameterTuner y especificamos los rangos de valores para los hiperparámetros utilizando ContinuousParameter, IntegerParameter o CategoricalParameter dependiendo de si se trata de un valor continuo, discreto o categórico; respectivamente.

In [ ]:
tuners = {}

total_jobs = 16
parallel_jobs = 2

tuners['GradientBoosting'] = HyperparameterTuner(
        estimator=estimators['GradientBoosting'],
        objective_metric_name=metric_name,
        objective_type='Maximize',
        hyperparameter_ranges={'learning-rate': ContinuousParameter(0.01, 0.1),
                                'min-samples-split': IntegerParameter(5, 15), 
                                'n-estimators': IntegerParameter(400, 800),
                                'max-depth': IntegerParameter(3, 10),
                                'max-features': IntegerParameter(15, 30)},
        metric_definitions=[{'Name': metric_name, 
                             'Regex': metric_regex}],
        max_jobs=total_jobs,
        max_parallel_jobs=parallel_jobs)

tuners['RandomForest'] = HyperparameterTuner(
        estimator=estimators['RandomForest'],
        objective_metric_name=metric_name,
        objective_type='Maximize',
        hyperparameter_ranges={'min-samples-split': IntegerParameter(3,10), 
                               'n-estimators': IntegerParameter(150,300),
                               'max-depth': IntegerParameter(20,35),
                               'max-features': IntegerParameter(15,30)},
        metric_definitions=[{'Name': metric_name, 
                              'Regex': metric_regex}],
        max_jobs=total_jobs,
        max_parallel_jobs=parallel_jobs)

tuners['ExtraTrees'] = HyperparameterTuner(
        estimator=estimators['ExtraTrees'],
        objective_metric_name=metric_name,
        objective_type='Maximize',
        hyperparameter_ranges={'min-samples-split': IntegerParameter(3,10), 
                               'n-estimators': IntegerParameter(150,350),
                               'max-depth': IntegerParameter(20,35),
                               'max-features': IntegerParameter(15,30)},
        metric_definitions=[{'Name': metric_name, 
                              'Regex': metric_regex}],
        max_jobs=total_jobs,
        max_parallel_jobs=parallel_jobs)


La variable total_jobs especifíca el número total de procesos de entrenamiento (combinaciones distintas de valores de hiperparámetros) a ejecutar y la variable parallel_jobs especifíca el número máximo de procesos a ejecutar en paralelo.

Para ejecutar cada uno de los procesos utilizamos el método fit() pasando como parámetros la ubicación de los datasets y mediante el parámetro wait=False es que le indicamos que ejecute el proceso de manera asíncrona (sin esperar a que cada uno de los procesos termine).

In [ ]:
for tuner in tuners:
    tuners[tuner].fit({'train_data': sagemaker_utils.get_processor_output_path(processor, 'train_data'),
                       'train_target': sagemaker_utils.get_processor_output_path(processor, 'train_target')}, 
                      job_name= f'{prefix}-{tuner}-{strftime("%M-%S", gmtime())}',
                      wait=False)


In [ ]:
#definimos un waiter

sagemaker_utils.wait_for_optmimization_jobs(tuners)

Mediante la invocación del método describe() podemos obtener el proceso de entrenamiento cuyo modelo resultante obtuvo el mejor desempeño, en este caso maximizando la métrica objetivo Recall. Posteriormente de los metadatos devueltos del mejor proceso de entrenamiento, podemos obtener los valores de los hiperparámetros así cómo el valor obtenido en la métrica objetivo.

In [ ]:
hyperparameters = {}
for tuner in tuners:
    best_training_job = tuners[tuner].describe()['BestTrainingJob']
    objective_metric = best_training_job['FinalHyperParameterTuningJobObjectiveMetric']
    
    hyperparameters[tuner] = best_training_job['TunedHyperParameters']
    print(tuner)
    print(f"\thyper parameters: {hyperparameters[tuner]}")
    print(f"\t{objective_metric['MetricName']}: {objective_metric['Value']}\n")


Ahora que tenemos el mejor candidato (modelo con mejor desempeño) para cada uno de los tres algoritmos, podemos pasar a comparar el desempeño entre estos para seleccionar el mejor